# Fahrradmessstellen Ergebnisse Radzählungen 2021

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re

In [2]:
#Messungen stündlich!!!
with open('ergebnissefahrradzaehlungen2021.csv', 'r', encoding='utf-8', errors='replace') as file:
    df = pd.read_csv(file, sep=';', header=None, low_memory=False)
    
#Sonderzeichen korrigieren
replacement_dict = {re.compile(r'�'): 'ü'}
for pattern, replacement in replacement_dict.items():
    df = df.replace(pattern, replacement, regex=True)

#Leere Zeilen entfernen
df = df.iloc[:, :-1]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.drop('5.12 BN - Straüburger Weg', axis=1)
df = df.reset_index(drop=True)

#Summen rausnehmen
summen_pro_uhrzeit = df.iloc[8760].tolist()
summen_pro_standort = df[' Summe'].tolist()
df = df.iloc[:-1, :-1]

#datetime index herstellen
df.set_index('Time', inplace=True)
month_mapping = {
    'Jan.': 'January',
    'Febr.': 'February',
    'Mrz.': 'March',
    'Apr.': 'April',
    'Mai': 'May',
    'Jun.': 'June',
    'Jul.': 'July',
    'Aug.': 'August',
    'Sept.': 'September',
    'Okt.': 'October',
    'Nov.': 'November',
    'Dez.': 'December'  
}
df.index = df.index.to_series().replace(month_mapping, regex=True)
df.index = pd.to_datetime(df.index, format='%d. %B %Y %H:%M')

df = df.drop(columns=['5.03 BN - Nordbrücke (Südseite)', '5.04 BN - Nordbrücke (Nordseite)', 
                      '5.05 BN - Südbrücke (Südseite)', '5.06 BN - Südbrücke (Nordseite)', 
                      '5.01 BN - Kennedybrücke (Nordseite)', '5.02 BN - Kennedybrücke (Südseite) Barometer', '5.10 BN - Brühltalweg'])


column_mapping = {
    '5.07 BN - Estermannufer': 'Estermannufer',
    '5.08 BN - Von-Sandt-Ufer': 'Von-Sandt-Ufer',
    '5.09 BN - Rhenusallee': 'Rhenusallee',
    '5.11 BN - Brühler Straüe': 'Brühler Straße',
    '5.14 BN - Mc Cloy Weg': 'Mc Cloy Weg',
    '5.15 BN - Weg auf Damm Neil': 'Weg auf Damm Neil',
    '5.13 BN - Wilhelm-Spiritus-Ufer': 'Wilhelm-Spiritus-Ufer'
}
order = ['Kennedybrücke', 'Nordbrücke', 'Südbrücke', 'Estermannufer', 'Von-Sandt-Ufer', 'Rhenusallee', 
         'Brühler Straße', 'Wilhelm-Spiritus-Ufer', 'Mc Cloy Weg', 'Weg auf Damm Neil']
df.rename(columns=column_mapping, inplace=True)
df = df[order]
df = df.applymap(lambda x: -1 if pd.isna(x) else x).astype(int)
df = df.resample('H').sum()
df

,Kennedybrücke,Nordbrücke,Südbrücke,Estermannufer,Von-Sandt-Ufer,Rhenusallee,Brühler Straße,Wilhelm-Spiritus-Ufer,Mc Cloy Weg,Weg auf Damm Neil
Time,,,,,,,,,,
2021-01-01 00:00:00,23,3,7,0,5,1,0,4,-1,1
2021-01-01 01:00:00,47,3,7,0,1,2,4,5,-1,4
2021-01-01 02:00:00,47,3,4,0,1,7,9,3,-1,4
2021-01-01 03:00:00,86,10,3,2,5,1,3,4,-1,1
2021-01-01 04:00:00,25,2,3,0,0,3,6,2,-1,3
...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,197,10,45,11,13,27,11,17,10,34
2021-12-31 20:00:00,55,12,17,5,10,10,5,9,30,12
2021-12-31 21:00:00,71,4,9,2,10,11,14,11,18,16


In [3]:
percentage_negative = ((df < 0).sum() / df.count()) * 100
percentage_negative

0
Kennedybrücke             0.000000
Nordbrücke                0.000000
Südbrücke                 0.000000
Estermannufer             3.082192
Von-Sandt-Ufer            0.000000
Rhenusallee               0.000000
Brühler Straße            0.331050
Wilhelm-Spiritus-Ufer     0.000000
Mc Cloy Weg              14.703196
Weg auf Damm Neil         0.000000
dtype: float64

## Auszüge

In [11]:
kennedy2021 = df['Kennedybrücke']
bonn2021_tag = df.sum(axis=1).resample('D').sum()
bonn2021_monat = bonn2021_tag.resample('M').sum()
%store kennedy2021 bonn2021_tag bonn2021_monat

Stored 'kennedy2021' (Series)
Stored 'bonn2021_tag' (Series)
Stored 'bonn2021_monat' (Series)


In [12]:
df.to_csv('df_Bonn2021.csv')
#bonn2021_tag.to_csv('Bonn2021_täglich.csv')